# AF de MEGADADOS

A prova tem duração de **2h30**. Veja mais informações no Blackboard!

Não se esqueça de **anexar o notebook** no **Blackboard** ao final da prova. Deixe as células **executadas**!

**NOME**: Gustavo E. S. Oliveira

## Insper autograding!

Para receber feedback dos exercício, iremos utilizar o `insper autograding`.

**Importante**: você precisará do `.env`, utilize o mesmo procedimento feito na aula de **Spark**!

In [1]:
!pip install -U git+https://github.com/macielcalebe/insperautograding.git
!pip install mysql-connector-python

  Cloning https://github.com/macielcalebe/insperautograding.git to /tmp/pip-req-build-y5sr43dh
  Running command git clone --filter=blob:none --quiet https://github.com/macielcalebe/insperautograding.git /tmp/pip-req-build-y5sr43dh
  Resolved https://github.com/macielcalebe/insperautograding.git to commit acdda51152774d9e1a979b426e41daa7a8a7793c
  Preparing metadata (setup.py) ... done


## Como resolver os exercícios?

Crie a base da prova em sua máquina (Exercício 1). Utilize o notebook, MySQL Workbench ou o conector para testar as queries e soluções. Quando estiver bastante certo de que a resposta está correta, faça a submissão para o servidor.

## Base de dados

Em alguns exercícios, iremos utilizar a base de dados `af_md_23_2`.

Execute o script `mensagens.sql` no Workbench para criar e popular a base.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [2]:
import mysql.connector
import os
import insperautograder.jupyter as ia
from functools import partial
from pyspark.sql import SparkSession
from dotenv import load_dotenv

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [3]:
load_dotenv(override=True)

def get_connection_helper(database):

    def run_db_query(connection, query, args=None):
        with connection.cursor() as cursor:
            results = cursor.execute(query, args, multi=True)
            for i, result in enumerate(results):
                if result.with_rows:
                    print(f"Resultado query {i}:")
                    for line in result.fetchall():
                        print(line)
                else:
                    print(f"Query {i} executada!")

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database=database,
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper("af_md_23_2")

DatabaseError: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (111)

In [4]:
spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext

### Notas

As primeiras cinco questões, que possuem correção automática, valem 60% da nota da prova.

Na API de correção automática a nota de cada questão será ponderada pelo seu peso. A nota será apresentada no intervalo 0 a 10, multiplique por 0.6 para saber a nota final considerando toda a prova.

Para conferir a nota da correção automática da prova, utilize:

In [5]:
ia.grades(task="af_md_23_2")

Defina os argumentos `subject` e `offering`!

In [6]:
ia.grades(by="TASK", task="af_md_23_2")

Defina os argumentos `subject` e `offering`!

**Exercício 1**:

O arquivo `timestamps.txt` contém uma lista de timestamps indicando quando foram feitos acessos a um sistema, um por linha. Cada timestamp é uma string no formato `YYYY-MM-DD HH:MM:SS`. Escreva uma função em Spark que recebe um rdd e um ano (nesta ordem) e retorna quantos acessos houveram em um determinado ano.

In [39]:
def total_acessos(rdd, ano):
    filtro = rdd.filter(lambda x: ano in x)
    qtd = filtro.count()
    return qtd

rdd = sc.textFile("timestamps.txt")

print(total_acessos(rdd, "2022"))
print(total_acessos(rdd, "2023"))

24985
25022


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [50]:
ia.sender(answer="total_acessos", task="af_md_23_2", question="ex01spark", answer_type="pycode")

interactive(children=(Button(description='Enviar ex01spark', style=ButtonStyle()), Output()), _dom_classes=('w…

**Exercício 2**:

Ainda no arquivo `timestamps.txt`, escreva uma função em Spark que recebe um rdd e retorna em ordem decrescente as 20 datas em que houveram mais acessos junto da quantidade de acessos em cada data.

In [40]:
def top20(rdd):
    # Passo 1: Mapear cada timestamp para uma tupla (data, 1)
    data_acesso_pairs = rdd.map(lambda x: (x.split(" ")[0], 1))
    
    # Passo 2: Reduzir por chave (data) para contar o número de acessos por data
    acessos_por_data = data_acesso_pairs.reduceByKey(lambda a, b: a + b)
    
    # Passo 3: Ordenar os resultados pela contagem de acessos em ordem decrescente
    acessos_ordenados = acessos_por_data.sortBy(lambda x: x[1], ascending=False)
    
    # Passo 4: Pegar as top 20 datas com mais acessos
    top_20 = acessos_ordenados.take(20)
    
    return top_20

top20(rdd)

[('2021-12-15', 161),
 ('2022-10-09', 154),
 ('2020-07-13', 151),
 ('2021-10-05', 150),
 ('2021-10-29', 149),
 ('2020-09-11', 148),
 ('2023-07-02', 148),
 ('2022-10-25', 148),
 ('2020-09-26', 147),
 ('2020-07-01', 147),
 ('2021-08-15', 147),
 ('2021-06-17', 145),
 ('2022-06-23', 144),
 ('2021-09-24', 144),
 ('2023-07-23', 143),
 ('2022-09-17', 143),
 ('2020-12-26', 143),
 ('2021-08-03', 142),
 ('2021-11-25', 142),
 ('2022-07-02', 142)]

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [ ]:
ia.sender(answer="total_acessos", task="af_md_23_2", question="ex02spark", answer_type="pycode")

**Exercício 3**:

Vimos na primeira parte do curso como podemos utilizar índices para acelerar a performance de *queries*. RDBMS como o MySQL suportam, inclusive, a construção de índices por múltiplas colunas.

Entretanto, em certas situações, como quando são realizadas primariamente buscar por igualdade em múltiplas colunas, uma alternativa é utilizar **hashing** de índices compostos. Nesta técnica, ao invés de construir um índice composto por múltiplas colunas e realizar as consultas neste formato:

```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.col1 = 'algum valor'
  AND t.col2 = 'outro valor'
  AND t.col3 = 'criterio de busca'
```

É construida manualmente uma coluna que contém o **hash da concatenação das colunas utilizadas na query**. Então, é criado um índice apenas na coluna que contém o hash. Assim, a *query* de consulta passa a ser, por exemplo:


```mysql
SELECT t.res1,
       t.res2
FROM tabela t
WHERE t.coluna_hash = MD5(CONCAT('algum valor', 'outro valor', 'criterio de busca'))
```

Neste exercício, você deve construir uma *trigger* para atualizar a coluna chamada `coluna_hash` da tabela de `mensagens` quando ocorrer um **update** na tabela de mensagens.

A tabela estará assim:
| id | col1    | col2     | col3    | mensagem                                 |
|----|---------|----------|---------|------------------------------------------|
| 1  | ANA     | SILVA    | ABACATE | Oi, bom dia                              |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |

Por exemplo, após criar a coluna `coluna_hash`, ao fazer um *update* no id `1`, ficará assim:

| id | col1    | col2     | col3    | mensagem                                 | coluna_hash                              |
|----|---------|----------|---------|------------------------------------------|------------------------------------------|
| 1  | ANA     | SILVA    | UVA     | Oi, bom dia                              | 2d0b14a194488b9080e080c8a7d82ef88fddbe1e |
| 2  | MARIA   | PEREIRA  | PERA    | Me liga quando puder                     |                                          |
| 3  | JOSE    | FERREIRA | ABACATE | Cade vc?                                 |                                          |
| 4  | ANTONIO | SILVA    | MACA    | Qual o status do pedido?                 |                                          |
| 5  | CAMILA  | WOK      | PERA    | Conseguiu analisar?                      |                                          |
| 6  | DANILO  | SCHINLE  | MACA    | Tudo certo pro jantar, vc vem?           |                                          |
| 7  | PEDRO   | PACKEL   | BANANA  | Complicado, vai ter que conferir com ela |                                          |
| 8  | MANOELA | NANTES   | PERA    | O lugar era lindo                        |                                          |
| 9  | CELSO   | ALIVERRI | ABACATE | A visao eh adequada                      |                                          |
| 10 | ELLEN   | WETLKS   | UVA     | Vamos juntos                             |                                          |

**Obs**:
- Execute o script `mensagens.sql` para construir a base de dados;
- Faça o DDL, em seu servidor local, para criar a coluna `coluna_hash` com array `CHAR` de tamanho 40. No servidor de testes esta coluna já existe e você não deve enviar esta parte;
- Imagine que o índice já existe na coluna de hash (não é relevante para o exercício que ele realmente exista);
- A coluna `coluna_hash` deve ser calculada com o uso da função `SHA1` a partir da concatenação dos colunas `col1`, `col2` e `col3` (nesta ordem e sem espaçadores).


In [15]:
sql_ex03 = """
CREATE TRIGGER IF NOT EXISTS atualiza_coluna_hash
BEFORE UPDATE ON mensagens
FOR EACH ROW
BEGIN
    SET NEW.coluna_hash = SHA1(CONCAT(NEW.col1, NEW.col2, NEW.col3));
END
"""

db(sql_ex03)

Query 0 executada!


Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [16]:
ia.sender(answer="sql_ex03", task="af_md_23_2", question="ex03trigger", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex03trigger', style=ButtonStyle()), Output()), _dom_classes=(…

**Exercício 04**:

Considere que foi criado o usuário `u_consult` com login a partir de qualquer host.

O usuário possui permissão de **LEITURA** e **INSERÇÃO** na tabela de `mensagens` utilizada no exercício anterior. Entretanto, ele não deveria poder **INSERIR** na tabela `mensagens`.

Faça as atualizações, revogando as permissões não mais necessárias.

In [18]:
sql_ex04 = """
REVOKE INSERT ON af_md_23_2.mensagens FROM 'u_consult'@'%'
"""

db(sql_ex04)

ProgrammingError: 1147 (42000): There is no such grant defined for user 'u_consult' on host '%' on table 'mensagens'

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [19]:
ia.sender(answer="sql_ex04", task="af_md_23_2", question="ex04perm", answer_type="pyvar")

interactive(children=(Button(description='Enviar ex04perm', style=ButtonStyle()), Output()), _dom_classes=('wi…

**Exercício 5**:

Um pesquisador irá analisar parte dos dados de um projeto científico para identificação de exoplanetas.

Os dados do projeto são armazenados em um servidor com quatro HDs de 10TB configurados em RAID 5.

Durante o projeto, surgiram questões como:

*"Considerando que são recebidos 14 GB por semana, quantos dias serão necessários para que se atinja 75% da capacidade disponível para armazenamento no servidor?"*

Para responder questões como esta rapidamente, você deve construir uma função `calc_dias_armazenamento` em python que recebe, nesta ordem:

- `tb_por_hd`: quantidade de terabytes de cada um dos quatro HDs;
- `gb_semana`: quantidade de novos dados recebidos a cada semana, em GB;
- `perc`: capacidade percentual limite que se quer analisar. O percentual será passado no intervalo `[0.0 ... 1.0]`.

Como resposta, você deve devolver a quantidade inteira de dias até que que a capacidade percentual seja atingida.

**Obs**:
- Considere o sistema internacional de unidades, ou seja, 1TB = 1000GB e 1GB=1000MB.

In [46]:
def calc_dias_armazenamento(tb_por_hd, gb_semana, perc):
    tb_total = tb_por_hd * 3
    gb_total = tb_total * 1000
    gb_disponivel = (gb_total*perc)
    gb_dia = gb_semana/7
    dias = int(gb_disponivel/gb_dia)
    return dias

In [47]:
# Alguns testes locais
assert calc_dias_armazenamento(10, 14000, 0.5) == 7
assert calc_dias_armazenamento(20, 12800, 0.9) == 29

Após testar localmente e considerar sua solução correta, faça o envio clicando no botão abaixo!

In [48]:
ia.sender(answer="calc_dias_armazenamento", task="af_md_23_2", question="ex05python", answer_type="pycode")

interactive(children=(Button(description='Enviar ex05python', style=ButtonStyle()), Output()), _dom_classes=('…

**Exercício 6**: (**Nota: 1,5**)

Na atividade da próxima *sprint*, foi atribuido ao seu time de analistas de negócio a entrega de um *Dashboard* para acompanhamento dos números de vendas da empresa (vendas por região, vendas por categoria de produto, vendas por canal, etc.)

Dado que isolamento é uma das propriedades fundamentais de bancos de dados relacionais e considerando que parte da especificação envolve:

- O acompanhamento será feito em tempo real;
- É ok ocorrerem imprecisões momentâneas.

Justifique **qual nível de isolamento de transações** você recomendaria que fosse utilizado no projeto.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

Para recomendar o nível de isolamento de transações para o projeto de dashboard de vendas em tempo real, é essencial entender os diferentes níveis de isolamento disponíveis em bancos de dados relacionais e como eles afetam o desempenho e a consistência dos dados. Os níveis de isolamento definidos pelo padrão SQL são:

1. **Read Uncommitted**
2. **Read Committed**
3. **Repeatable Read**
4. **Serializable**

### Read Uncommitted
- **Descrição:** Permite ler dados que foram modificados por outras transações, mas ainda não foram confirmados (commit). Isso pode resultar em leituras sujas (dirty reads).
- **Vantagem:** Máxima performance devido à mínima restrição de bloqueio.
- **Desvantagem:** Pode causar inconsistências devido a leituras de dados não confirmados.

### Read Committed
- **Descrição:** Garante que uma transação só leia dados que foram confirmados. Isso evita leituras sujas.
- **Vantagem:** Balanceia bem a consistência dos dados com a performance.
- **Desvantagem:** Pode causar leituras não repetíveis, onde um valor lido pode ser alterado por outra transação antes da conclusão da transação atual.

### Repeatable Read
- **Descrição:** Garante que, uma vez lidos, os dados não podem ser modificados por outras transações até que a transação atual termine. Isso evita leituras não repetíveis.
- **Vantagem:** Oferece uma alta consistência dos dados.
- **Desvantagem:** Pode levar a bloqueios de leitura e escrita, reduzindo a performance.

### Serializable
- **Descrição:** É o nível de isolamento mais alto, onde as transações são completamente isoladas umas das outras. Garante a serialização completa das transações.
- **Vantagem:** Máxima consistência dos dados.
- **Desvantagem:** Pode causar alta contenção de bloqueios, resultando em baixa performance.

### Recomendação: Read Committed

**Justificativa:**
Para o projeto de dashboard de vendas em tempo real, a prioridade é fornecer dados atualizados de forma rápida e eficiente, aceitando pequenas imprecisões momentâneas. O nível de isolamento **Read Committed** é o mais adequado para essa situação devido às seguintes razões:

1. **Balanceamento entre Consistência e Performance:**
   - **Consistência:** Garante que apenas dados confirmados sejam lidos, evitando leituras sujas. Isso é importante para garantir que o dashboard não mostre informações totalmente incorretas ou intermediárias que ainda não foram confirmadas.
   - **Performance:** Menos rigoroso que níveis mais altos como Repeatable Read e Serializable, reduzindo a contenção de bloqueios e permitindo um desempenho aceitável para operações em tempo real.

2. **Adequação ao Cenário:**
   - **Imprecisões Momentâneas:** O nível de isolamento Read Committed permite que, entre leituras, alguns dados possam ser atualizados por outras transações, resultando em leituras não repetíveis. No contexto do dashboard, pequenas variações nos números de vendas são aceitáveis e não impactam a funcionalidade principal do sistema.
   - **Carga de Trabalho:** Dashboards em tempo real geralmente exigem leituras frequentes e rápidas. Níveis de isolamento mais altos poderiam introduzir latência desnecessária, impactando negativamente a experiência do usuário.

3. **Exemplos de Uso:**
   - Em sistemas de monitoramento onde a última informação disponível é mais valiosa do que a consistência perfeita.
   - Em relatórios de vendas onde pequenas discrepâncias momentâneas são toleráveis em troca de respostas mais rápidas.

### Conclusão
Recomendo o nível de isolamento **Read Committed** para o projeto de dashboard de vendas em tempo real. Este nível oferece um bom equilíbrio entre a consistência dos dados e a performance, atendendo às necessidades de monitoramento contínuo com atualizações rápidas e aceitando imprecisões momentâneas que são aceitáveis no contexto descrito.

</div>

**Exercício 7**:

Considere um cenário onde projetos são propostos por empresas em busca de financiamento. Cada projeto é proposto por apenas uma empresa, mas a empresa pode propor muitos projetos.

Os projetos podem ou não receber apoio financeiro. Esta decisão é tomada por meio de uma votação, onde usuários membros de um grupo qualquer decidirão se apoiam ou não cada projeto. Cada usuário vota apenas uma vez em cada projeto (`S: sim`, `N: não`, `A: anula`).

Considerando esta tabela de `votos`:

| id_usuario(PK) | nome           | id_projeto(PK) | projeto     | descricao                              | id_empresa | empresa | voto |
|----------------|----------------|----------------|-------------|----------------------------------------|------------|---------|------|
| 20             | Antonio Soares | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 21             | Marina Pereira | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 22             | Camila Mert    | 1              | Sockerator  | Emparelhador de meias                  | 50         | UniX    | N    |
| 20             | Antonio Soares | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | S    |
| 22             | Camila Mert    | 2              | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | N    |
| 20             | Antonio Soares | 3              | LazyHat     | Um boné com apoio para petiscos        | 50         | UniX    | A    |
| 21             | Marina Pereira | 3              | Lazy Hat    | Um boné com apoio para petiscos        | 50         | UniX    | A    |

Chamamos de normalização o processo de rearranjar o banco de dados para que obedeça às várias formas normais. Assim, reduzimos a redundância, aumentamos o desempenho e a integridade de dados.

**a)** (**Nota: 1,0**) Ao analisar a tabela, um aluno fez a seguinte afirmação:

*"A tabela de `votos` está na `2NF` mas não está na `1NF`"*

A afirmação do aluno está correta? Justifique detalhadamente.

Esta questão será corrigida considerando a seguinte rubrica:

| Conceito | Nota | Descrição                                                                                                                                               |
|:----------:|----------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------|
| I        | 0.0 |Apenas citou o assunto ou alguns fatos sem explicações                                                                                                  |
| D        | 0.3 |Explicou superficialmente o assunto ou fatos mas sem muitos detalhes conclusivos                                                                        |
| C        | 0.6 |Explicou com detalhes, apresentando definições concretas.                                                                                               |
| B        | 0.8 |Explicou com detalhes, apresentando definições concretas e exemplos de uso.                                                                            |
| A        | 1.0 |Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles. |

<div class="alert alert-success">

A afirmação do aluno de que a tabela de `votos` está na `2NF` mas não está na `1NF` é incorreta. Vamos analisar o porquê, abordando os conceitos de normalização de bancos de dados, especificamente as primeiras e segundas formas normais (1NF e 2NF).

### Primeira Forma Normal (1NF)

Para que uma tabela esteja na 1NF, ela deve atender aos seguintes requisitos:

1. **Atomicidade**: Todos os valores em uma coluna devem ser atômicos, ou seja, indivisíveis.
2. **Chave Primária**: A tabela deve ter uma chave primária, que identifica unicamente cada linha.
3. **Ausência de Colunas Repetitivas**: Não deve haver grupos de colunas repetitivas.

Vamos verificar se a tabela `votos` atende a esses critérios:

- **Atomicidade**: Todos os valores na tabela `votos` são atômicos. Por exemplo, cada coluna (`id_usuario`, `nome`, `id_projeto`, `projeto`, `descricao`, `id_empresa`, `empresa`, `voto`) contém valores indivisíveis.
- **Chave Primária**: A combinação de `id_usuario` e `id_projeto` pode ser considerada uma chave composta que identifica unicamente cada linha, garantindo unicidade.
- **Ausência de Colunas Repetitivas**: Não há grupos de colunas repetitivas na tabela.

Portanto, a tabela `votos` está na 1NF.

### Segunda Forma Normal (2NF)

Para que uma tabela esteja na 2NF, ela deve atender aos seguintes requisitos:

1. **Estar na 1NF**: A tabela deve estar na 1NF.
2. **Ausência de Dependências Parciais**: Todos os atributos não chave devem ser totalmente dependentes da chave primária inteira e não apenas de uma parte dela.

Vamos verificar se a tabela `votos` atende a esses critérios:

- **Dependências Parciais**: A chave primária composta é `(id_usuario, id_projeto)`. Devemos verificar se todos os atributos não chave dependem dessa chave composta inteira.

    - `nome` depende apenas de `id_usuario`.
    - `projeto` e `descricao` dependem apenas de `id_projeto`.
    - `id_empresa` e `empresa` dependem apenas de `id_projeto`.

Como há dependências parciais (atributos que dependem apenas de parte da chave primária), a tabela `votos` não está na 2NF.

### Conclusão

A tabela `votos` está na 1NF, mas não está na 2NF devido à existência de dependências parciais. Portanto, a afirmação do aluno está incorreta, pois a tabela não pode estar na 2NF sem antes estar na 1NF. A tabela `votos` precisa passar por uma normalização adicional para eliminar essas dependências parciais e atender aos critérios da 2NF.

**Rubrica de Correção:**

- **Conceito A (1.0)**: Explicou com detalhes, apresentando definições concretas, exemplos de uso e ainda outros tópicos correlatos, fazendo uma conexão lógica entre eles.
- **Explicação Detalhada**: Definições de 1NF e 2NF, análise da tabela `votos` em relação a esses critérios, explicação das dependências parciais e correlação entre as formas normais.


</div>

**b**) (**Nota: 1,5**) Proponha modificações até que esteja na `3NF`.

**Obs**:
- Faça um passo a passo, deixando claro sua motivação em cada etapa.
- Você não deve gerar SQL, apenas texto e tabelas markdown!

<div class="alert alert-success">

Para normalizar a tabela `votos` até a Terceira Forma Normal (3NF), vamos seguir um passo a passo, abordando as motivações em cada etapa.

### Passo 1: Tabela Inicial

A tabela inicial `votos` está na 1NF, mas não na 2NF ou 3NF. Aqui está a tabela original:

| id_usuario (PK) | nome           | id_projeto (PK) | projeto     | descricao                              | id_empresa | empresa | voto |
|-----------------|----------------|-----------------|-------------|----------------------------------------|------------|---------|------|
| 20              | Antonio Soares | 1               | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 21              | Marina Pereira | 1               | Sockerator  | Emparelhador de meias                  | 50         | UniX    | S    |
| 22              | Camila Mert    | 1               | Sockerator  | Emparelhador de meias                  | 50         | UniX    | N    |
| 20              | Antonio Soares | 2               | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | S    |
| 22              | Camila Mert    | 2               | Snacktapult | Lança petiscos diretamente para a boca | 51         | Abx     | N    |
| 20              | Antonio Soares | 3               | LazyHat     | Um boné com apoio para petiscos        | 50         | UniX    | A    |
| 21              | Marina Pereira | 3               | Lazy Hat    | Um boné com apoio para petiscos        | 50         | UniX    | A    |

### Passo 2: Eliminar Dependências Parciais (2NF)

Para atingir a 2NF, precisamos remover dependências parciais. Isso significa que todos os atributos não chave devem depender da chave primária inteira. Vamos criar tabelas separadas para `projetos`, `empresas` e `usuarios`.

#### Tabela de Projetos

| id_projeto (PK) | projeto     | descricao                              | id_empresa |
|-----------------|-------------|----------------------------------------|------------|
| 1               | Sockerator  | Emparelhador de meias                  | 50         |
| 2               | Snacktapult | Lança petiscos diretamente para a boca | 51         |
| 3               | LazyHat     | Um boné com apoio para petiscos        | 50         |

#### Tabela de Empresas

| id_empresa (PK) | empresa |
|-----------------|---------|
| 50              | UniX    |
| 51              | Abx     |

#### Tabela de Usuários

| id_usuario (PK) | nome           |
|-----------------|----------------|
| 20              | Antonio Soares |
| 21              | Marina Pereira |
| 22              | Camila Mert    |

### Passo 3: Tabela de Votos

Agora, a tabela `votos` contém apenas as chaves primárias e o voto, removendo as dependências parciais:

| id_usuario (PK) | id_projeto (PK) | voto |
|-----------------|------------------|------|
| 20              | 1                | S    |
| 21              | 1                | S    |
| 22              | 1                | N    |
| 20              | 2                | S    |
| 22              | 2                | N    |
| 20              | 3                | A    |
| 21              | 3                | A    |

### Passo 4: Eliminar Dependências Transitivas (3NF)

Para atingir a 3NF, precisamos garantir que todos os atributos não chave dependam apenas da chave primária, não de outros atributos não chave. Já fizemos isso ao dividir a tabela em três tabelas separadas. Agora vamos verificar as tabelas resultantes:

#### Tabela de Projetos (já está na 3NF)

| id_projeto (PK) | projeto     | descricao                              | id_empresa |
|-----------------|-------------|----------------------------------------|------------|
| 1               | Sockerator  | Emparelhador de meias                  | 50         |
| 2               | Snacktapult | Lança petiscos diretamente para a boca | 51         |
| 3               | LazyHat     | Um boné com apoio para petiscos        | 50         |

Nesta tabela, `projeto` e `descricao` dependem apenas de `id_projeto`, e `id_empresa` está relacionado diretamente ao projeto.

#### Tabela de Empresas (já está na 3NF)

| id_empresa (PK) | empresa |
|-----------------|---------|
| 50              | UniX    |
| 51              | Abx     |

Nesta tabela, `empresa` depende apenas de `id_empresa`.

#### Tabela de Usuários (já está na 3NF)

| id_usuario (PK) | nome           |
|-----------------|----------------|
| 20              | Antonio Soares |
| 21              | Marina Pereira |
| 22              | Camila Mert    |

Nesta tabela, `nome` depende apenas de `id_usuario`.

#### Tabela de Votos (já está na 3NF)

| id_usuario (PK) | id_projeto (PK) | voto |
|-----------------|------------------|------|
| 20              | 1                | S    |
| 21              | 1                | S    |
| 22              | 1                | N    |
| 20              | 2                | S    |
| 22              | 2                | N    |
| 20              | 3                | A    |
| 21              | 3                | A    |

Nesta tabela, `voto` depende apenas da combinação da chave primária composta `id_usuario` e `id_projeto`.

### Conclusão

Com estas modificações, garantimos que todas as tabelas estão na 3NF. Cada atributo não chave é totalmente dependente da chave primária inteira, e não há dependências transitivas. A nova estrutura reduz a redundância, melhora a integridade dos dados e facilita a manutenção do banco de dados.
    
Crie quantas tabelas precisar!

Dica: você pode editar tabelas markdown em https://www.tablesgenerator.com/markdown_tables
</div>

### Conferindo as notas!

In [56]:
ia.grades(task="af_md_23_2")

|    | Atividade   | Exercício   |   Peso |   Nota |
|---:|:------------|:------------|-------:|-------:|
|  0 | af_md_23_2  | ex01spark   |      2 |     10 |
|  1 | af_md_23_2  | ex02spark   |      3 |     10 |
|  2 | af_md_23_2  | ex03trigger |      3 |     10 |
|  3 | af_md_23_2  | ex04perm    |      1 |     10 |
|  4 | af_md_23_2  | ex05python  |      2 |     10 |

In [57]:
ia.grades(by="TASK", task="af_md_23_2")

|    | Tarefa     |   Nota |
|---:|:-----------|-------:|
|  0 | af_md_23_2 |     10 |

## Entrega!

É hora de entregar. Faça `commit` e finalize o teste no proctorio!